In [5]:
import os
import pyrealsense2 as rs
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pyocr
import pyocr.builders
import glob

In [108]:
def cv2pil(image):
    new_image = image.copy()
    if new_image.ndim == 2:  # モノクロ
        pass
    elif new_image.shape[2] == 3:  # カラー
        new_image = new_image[:, :, ::-1]
    elif new_image.shape[2] == 4:  # 透過
        new_image = new_image[:, :, [2, 1, 0, 3]]
    new_image = Image.fromarray(new_image)
    return new_image

def pil2cv(image):
    ''' PIL型 -> OpenCV型 '''
    new_image = np.array(image, dtype=np.uint8)
    if new_image.ndim == 2:  # モノクロ
        pass
    elif new_image.shape[2] == 3:  # カラー
        new_image = new_image[:, :, ::-1]
    elif new_image.shape[2] == 4:  # 透過
        new_image = new_image[:, :, [2, 1, 0, 3]]
    return new_image

In [10]:
config = rs.config()

config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)

In [11]:
tools = pyocr.get_available_tools()
tool = tools[0]

In [135]:
# OCRをしたい画像のファイルパスを書く
img = Image.open('../../realsense_0727/image_folder/1595910350_color.jpg')

In [170]:
# wordboxがついた画像を出力
builder = pyocr.builders.WordBoxBuilder(tesseract_layout=6)
result = tool.image_to_string(img, lang='jpn', builder=builder)

out = cv2.imread('../../realsense_0727/image_folder/1595910477_color.jpg')
for d in result:
    cv2.rectangle(out, d.position[0], d.position[1], (0, 0, 255), 2)
    
cv2.imwrite('ツナマヨ.jpg', out)

True

In [179]:
# OCRの結果を出力
img = Image.open('../../realsense_0727/image_folder/1595910477_color.jpg')
img = img.resize((img.width*2, img.height*2))
#img = img.rotate(-7)
img = pil2cv(img)
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#ret3, th3 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
threshold = 110
ret, img_thresh = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
img = cv2pil(img_thresh)
builder = pyocr.builders.TextBuilder(tesseract_layout=6)
result = tool.image_to_string(img, lang='jpn+eng', builder=builder)
print(result)

H
uu
HELL ha
. pa, 10a 0 0
i EE
1 』 。
|


In [180]:
img.show()

<h1>すべてを実行する</h1>

<h2>OCRにかける。文字を出力する。</h2>

In [78]:
pipeline = rs.pipeline()
profile = pipeline.start(config)
align = align = rs.align(rs.stream.color)
count = 0
try:
    while True:
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)

        color_frame = aligned_frames.get_color_frame()
        depth_frame = aligned_frames.get_depth_frame()
        if not depth_frame or not color_frame:
            continue

        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        color_image = color_image[:, :, [2, 1, 0]]

        count += 1
        if count == 15:
            break                         
finally:
    pipeline.stop()
    
gray = cv2.cvtColor(color_image, cv2.COLOR_RGB2GRAY)
#th3 = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
#ret3,th3 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
threshold = 80

ret, img_thresh = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
img = cv2pil(img_thresh)
builder = pyocr.builders.TextBuilder(tesseract_layout=6)
result = tool.image_to_string(img, lang='jpn', builder=builder)

print(result)

m で で 回 記
時 中 か は は 各
上 導
抗菌 イメ の

Ne

@@⑳


In [80]:
img.show()

<h2>OCRにかける、バウンディングボックスを出力</h2>

In [39]:
pipeline = rs.pipeline()
profile = pipeline.start(config)
align = align = rs.align(rs.stream.color)
count = 0
try:
    while True:
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)

        color_frame = aligned_frames.get_color_frame()
        depth_frame = aligned_frames.get_depth_frame()
        if not depth_frame or not color_frame:
            continue

        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        color_image = color_image[:, :, [2, 1, 0]]

        count += 1
        if count == 15:
            break                         
finally:
    pipeline.stop()
    
cv2.imwrite('test.jpg', color_image)
gray = cv2.cvtColor(color_image, cv2.COLOR_RGB2GRAY)
ret3, th3 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
img = cv2pil(th3)
builder = pyocr.builders.WordBoxBuilder(tesseract_layout=6)
result = tool.image_to_string(img, lang='jpn', builder=builder)

img = cv2.imread('test.jpg')
for d in result:
    cv2.rectangle(img, d.position[0], d.position[1], (0, 0, 255), 2)
    
pil_out = cv2pil(img)
pil_out.show()